In [1]:
import numpy as np
from typing import Dict, List, Tuple, Any, Set
from openai import OpenAI
import os
import re
import sys
import importlib
from datetime import datetime
from src.pipeline.hyper_heuristics.single import SingleHyperHeuristic
from src.util.llm_client.get_llm_client import get_llm_client
from src.util.util import search_file
from src.run_hyper_heuristic.run_hyper_heuristic import run_hyper_heuristic, evaluate_all_heuristics
from src.run_hyper_heuristic.helper_function import select_complementary_pair, load_heuristic_code, extract_code_from_response, save_generated_heuristic, select_parent_for_local_search
from src.run_hyper_heuristic.prompt_cs import CSPromptFormatter, generate_complementary_heuristic, call_deepseek_reasoner
from src.run_hyper_heuristic.prompt_ls import LSPromptFormatter, generate_complementary_ls_heuristic
from src.run_hyper_heuristic.prompt_ie import IEPromptFormatter, generate_complementary_ie_heuristic
from src.run_hyper_heuristic.smoke_test import SmokeTestRunner,standalone_smoke_test
from src.run_hyper_heuristic.complete_cs_workflow import complete_cs_workflow
from src.run_hyper_heuristic.local_search_workflow import local_search_workflow
from src.run_hyper_heuristic.improve_efficiency import improve_efficiency
import json
import random

In [4]:
class EoHS:
    """Evolution of Heuristic Set"""
    
    def __init__(self, 
                 problem: str,
                 api_key: str,
                 heuristic_dir: str,
                 task_description_file: str,
                 output_dir: str,
                 model: str,
                 population_size: int,
                 max_evaluations: int,
                 time_limit: int):
        
        self.problem = problem
        self.api_key = api_key
        self.heuristic_dir = heuristic_dir
        self.task_description_file = task_description_file
        self.output_dir = output_dir
        self.n = population_size
        self.Nmax = max_evaluations
        self.model = model
        self.evaluations = 0
        self.llm_config_file = os.path.join("output", "llm_config", "azure_gpt_4o.json")  
        self.iterations_scale_factor = 1.0 
        self.result_dir = "result"
        self.time_limit = time_limit
        
        # 用于跟踪已选择的互补对
        self.selected_pairs: Set[Tuple[str, str]] = set()
       
        
    def initialize_population(self, results_dict: Dict) -> List[Dict]:
        """初始化种群 P0"""
        population = []
        for heuristic_name, heuristic_scores in results_dict.items():
            heuristic_info = {
                'name': heuristic_name,
                'performance_vector': heuristic_scores,
                'avg_performance': np.mean(heuristic_scores)
            }
            population.append(heuristic_info)
        return population
    
    def memetic_search(self, population: List[Dict], results_dict: Dict, time_limit) -> List[Dict]:
        """
        互补感知的Memetic搜索
        目标：生成 n 个新启发式
        """
        new_heuristics = []
        attempts = 0
        max_attempts = self.n * 5  # 允许一些失败尝试
        
        print(f"\n🔄 开始Memetic Search，目标生成 {self.n} 个新启发式")
        
        while len(new_heuristics) < self.n and attempts < max_attempts:
            attempts += 1
            
            # 50%概率使用CS，50%使用LS
            use_cs = random.random() < 0.5
            
            if use_cs:
                # Complementary-aware Search
                try:
                    print(f"\n  🔍 尝试 CS #{len(new_heuristics)+1}/{self.n} (尝试 {attempts}/{max_attempts})")
                    
                    file_path, code = complete_cs_workflow(
                        results_dict=results_dict,
                        api_key=self.api_key,
                        problem=self.problem,
                        heuristic_dir=self.heuristic_dir,
                        task_description_file=self.task_description_file,
                        output_dir=self.output_dir,
                        model=self.model
                    )
                    
                    # 烟雾测试
                    if standalone_smoke_test(file_path, api_key=self.api_key):
                        # 评估新启发式
                        perf_vector, avg_perf = self.evaluate_heuristic(file_path,time_limit)
                        feability = True
                        penalty_value = 1e10-1
                        for h_each_result in perf_vector:
                            if h_each_result > penalty_value:
                                feability = False 
                        if feability:
                            print(f" 加入该新算法: {file_path}")
                            new_heuristics.append({
                                'name': file_path,
                                'performance_vector': perf_vector,
                                'avg_performance': avg_perf
                            })
                            print(f"    ✅ CS 成功: {file_path} (avg: {avg_perf:.4f})")
                            self.evaluations += 1
                        else:
                            print('需要计算时间效率提升')
                            file_path, code = improve_efficiency(
                                                file_path=file_path,
                                                api_key=self.api_key,
                                                problem=self.problem,
                                                heuristic_dir=self.heuristic_dir,
                                                task_description_file=self.task_description_file,
                                                output_dir=self.output_dir,
                                                model=self.model
                                            )
                            # 烟雾测试
                            if standalone_smoke_test(file_path, api_key=self.api_key):
                                # 评估新启发式
                                perf_vector, avg_perf = self.evaluate_heuristic(file_path,time_limit)
                                feability = True
                                penalty_value = 1e10-1
                                for h_each_result in perf_vector:
                                    if h_each_result > penalty_value:
                                        feability = False 
                                if feability:
                                    print(f" 加入该新算法: {file_path}")
                                    new_heuristics.append({
                                        'name': file_path,
                                        'performance_vector': perf_vector,
                                        'avg_performance': avg_perf
                                    })
                                    print(f"    ✅ CS 成功: {file_path} (avg: {avg_perf:.4f})")
                                    self.evaluations += 1
                                else:
                                    print('提升计算时间效率失败，放弃该算法')

                            else:
                                print(f"    ❌ CS 烟雾测试失败")
                    else:
                        print(f"    ❌ CS 烟雾测试失败")
                        
                except Exception as e:
                    print(f"    ❌ CS 失败: {e}")
                    
            else:
                # Local Search
                try:
                    print(f"\n  🔧 尝试 LS #{len(new_heuristics)+1}/{self.n} (尝试 {attempts}/{max_attempts})")
                    
                    file_path, code = local_search_workflow(
                        results_dict=results_dict,
                        api_key=self.api_key,
                        problem=self.problem,
                        heuristic_dir=self.heuristic_dir,
                        task_description_file=self.task_description_file,
                        output_dir=self.output_dir,
                        model=self.model
                    )
                    
                    if standalone_smoke_test(file_path, api_key=self.api_key):
                        # 评估新启发式
                        perf_vector, avg_perf = self.evaluate_heuristic(file_path,time_limit)
                        feability = True
                        penalty_value = 1e10-1
                        for h_each_result in perf_vector:
                            if h_each_result > penalty_value:
                                feability = False 
                        if feability:
                            print(f" 加入该新算法: {file_path}")
                            new_heuristics.append({
                                'name': file_path,
                                'performance_vector': perf_vector,
                                'avg_performance': avg_perf
                            })
                            print(f"    ✅ CS 成功: {file_path} (avg: {avg_perf:.4f})")
                            self.evaluations += 1
                        else:
                            print('需要计算时间效率提升')
                            file_path, code = improve_efficiency(
                                                file_path=file_path,
                                                api_key=self.api_key,
                                                problem=self.problem,
                                                heuristic_dir=self.heuristic_dir,
                                                task_description_file=self.task_description_file,
                                                output_dir=self.output_dir,
                                                model=self.model
                                            )
                            if standalone_smoke_test(file_path, api_key=self.api_key):
                                # 评估新启发式
                                perf_vector, avg_perf = self.evaluate_heuristic(file_path,time_limit)
                              
                                feability = True
                                penalty_value = 1e10-1
                                for h_each_result in perf_vector:
                                    if h_each_result > penalty_value:
                                        feability = False 
                                if feability:
                                    print(f" 加入该新算法: {file_path}")
                                    new_heuristics.append({
                                        'name': file_path,
                                        'performance_vector': perf_vector,
                                        'avg_performance': avg_perf
                                    })
                                    print(f"    ✅ CS 成功: {file_path} (avg: {avg_perf:.4f})")
                                    self.evaluations += 1
                                else:
                                    print('提升计算时间效率失败，放弃该算法')

                            else:
                                print(f"    ❌ LS 烟雾测试失败")
                    else:
                        print(f"    ❌ LS 烟雾测试失败")
                        
                except Exception as e:
                    print(f"    ❌ LS 失败: {e}")
            
            # 检查是否达到最大评估次数
            if self.evaluations >= self.Nmax:
                print(f"\n⚠️  达到最大评估次数 {self.Nmax}，停止生成")
                break
        
        print(f"\n📊 Memetic Search 完成: 成功生成 {len(new_heuristics)}/{self.n} 个新启发式")
        return new_heuristics
    
    def complementary_population_management(self, 
                                            current_pop: List[Dict], 
                                            new_pop: List[Dict]) -> List[Dict]:
        """
        互补种群管理 (CPM)
        从 2n 个候选中选择 n 个
        """
        # 合并候选池: 最多 2n 个候选
        candidates = current_pop + new_pop
        candidate_pool_size = len(candidates)
        
        print(f"\n🔍 种群管理: 从 {candidate_pool_size} 个候选中选择 {self.n} 个")
        
        if not candidates:
            print("  ⚠️  候选池为空！")
            return current_pop
        
        m = len(candidates[0]['performance_vector'])  # 实例数量
        
        # 1. 选择平均性能最好的作为第一个
        best_candidate = min(candidates, key=lambda h: h['avg_performance'])
        selected = [best_candidate]
        candidates = [h for h in candidates if h['name'] != best_candidate['name']]
        
        print(f"  1️⃣  初始选择: {best_candidate['name']} (avg: {best_candidate['avg_performance']:.4f})")
        
        # 2. 迭代选择剩余 n-1 个启发式
        for step in range(min(self.n - 1, len(candidates))):
            if not candidates:
                print(f"  ⚠️  候选池已空，仅选择了 {len(selected)}/{self.n} 个")
                break
            
            # 计算每个候选的 Delta CPI
            best_delta = -np.inf
            best_candidate = None
            
            for candidate in candidates:
                delta_cpi = self.compute_delta_cpi(selected, candidate, m)
                if delta_cpi > best_delta:
                    best_delta = delta_cpi
                    best_candidate = candidate
            
            if best_candidate:
                selected.append(best_candidate)
                candidates = [h for h in candidates if h['name'] != best_candidate['name']]
                print(f"  {step+2}️⃣  选择: {best_candidate['name']} "
                      f"(avg: {best_candidate['avg_performance']:.4f}, Δ CPI: {best_delta:.4f})")
        
        print(f"  ✅ 最终选择 {len(selected)} 个启发式")
        return selected
    
    def compute_delta_cpi(self, current_set: List[Dict], candidate: Dict, m: int) -> float:
        """计算Delta CPI - 公式(5)"""
        delta = 0.0
        
        for j in range(m):
            # f*_Hk,ij: 当前集合在实例j上的最佳得分
            f_star = min(h['performance_vector'][j] for h in current_set)
            
            # fij(candidate): 候选启发式在实例j上的得分
            f_candidate = candidate['performance_vector'][j]
            
            # max(f*_Hk,ij - fij(hi), 0)
            delta += max(f_star - f_candidate, 0)
        
        return delta
    
    def evaluate_heuristic(self, heuristic_path: str,time_limit) -> Tuple[List[float], float]:
        """评估启发式在所有实例上的性能"""
        results = run_hyper_heuristic(
            problem=self.problem,
            heuristic=heuristic_path,
            test_data="test_data",
            llm_config_file=self.llm_config_file,
            heuristic_dir=self.heuristic_dir,
            iterations_scale_factor=self.iterations_scale_factor,
            result_dir=self.result_dir,
            time_limit=time_limit
        )
        
        performance_vector = results
        avg_performance = np.mean(performance_vector)
        
        return performance_vector, avg_performance
    
    def run(self, initial_results_dict: Dict,time_limit) -> List[Dict]:
        """主进化循环"""
        print("\n" + "="*80)
        print("🚀 EoH-S 算法开始")
        print("="*80)
        print(f"配置: 种群大小={self.n}, 最大评估次数={self.Nmax}")
        
        # 1. 初始化种群 P0
        population = self.initialize_population(initial_results_dict)
        print(f"\n✓ 初始化种群: {len(population)} 个启发式")
        
        # 2. 进化循环
        iteration = 0
        while self.evaluations < self.Nmax:
            iteration += 1
            print(f"\n{'='*80}")
            print(f"📈 第 {iteration} 代进化 (已评估: {self.evaluations}/{self.Nmax})")
            print(f"{'='*80}")
            
            # (a) Memetic Search: 生成 n 个新启发式
            new_heuristics = self.memetic_search(population, initial_results_dict,time_limit)
            
            # 如果没有生成任何新启发式，提前终止
            if not new_heuristics:
                print("\n⚠️  未能生成新启发式，进化终止")
                break
            
            # (b) Population Management: 从 2n 候选中选择 n 个
            # 注意：当前种群 n 个 + 新生成 n 个 = 最多 2n 个候选
            population = self.complementary_population_management(population, new_heuristics)
            
            # 计算当前 CPI
            current_cpi = self.compute_cpi(population)
            print(f"\n📊 当前种群 CPI: {current_cpi:.4f}")
            
            # 检查是否达到最大评估次数
            if self.evaluations >= self.Nmax:
                print(f"\n✅ 达到最大评估次数 {self.Nmax}，进化结束")
                break
        
        print("\n" + "="*80)
        print("🎉 EoH-S 算法完成!")
        print("="*80)
        
        return population
    
    def compute_cpi(self, population: List[Dict]) -> float:
        """计算互补性能指标 (CPI) - 公式(2)"""
        if not population:
            return float('inf')
        
        m = len(population[0]['performance_vector'])
        total = 0.0
        
        for j in range(m):
            # 每个实例上的最佳得分
            f_star = min(h['performance_vector'][j] for h in population)
            total += f_star
        
        return total / m


# 使用示例
def main():
    # 配置
    problem = "tsp"
    api_key = "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"
    model = "x-ai/grok-code-fast-1"
    heuristic_dir = "basic_heuristics"
    task_description_file = fr"\src\problems\{problem}\task_description.txt"
    output_dir = fr"\src\problems\{problem}\heuristics\basic_heuristics"
    test_data="test_data"
    result_dir="result"
    iterations_scale_factor=1.0
    save_to_file=True
    
    print("="*80)
    print("📋 步骤 1: 评估初始启发式")
    print("="*80)
    
    time_limit = 10*60
    
    # 评估初始启发式
    results_dict = evaluate_all_heuristics(
        problem=problem,
        heuristic_dir=heuristic_dir,
        test_data=test_data,
        iterations_scale_factor=iterations_scale_factor,
        result_dir=result_dir,
        save_to_file=save_to_file,
        time_limit=time_limit
    )
    
    print(results_dict)
    
    print("\n" + "="*80)
    print("📋 步骤 2: 运行 EoH-S")
    print("="*80)
    
    # 运行 EoH-S
    eohs = EoHS(
        problem=problem,
        api_key=api_key,
        heuristic_dir=heuristic_dir,
        task_description_file=task_description_file,
        output_dir=output_dir,
        model = model,
        population_size=5,  
        max_evaluations=100,
        time_limit=time_limit
    )
    
    final_population = eohs.run(results_dict,time_limit)
    
    # 输出最终结果
    print("\n" + "="*80)
    print("📊 最终结果")
    print("="*80)
    print(f"🎯 最终启发式集大小: {len(final_population)}")
    print(f"🏆 最终 CPI: {eohs.compute_cpi(final_population):.4f}")
    
    print("\n📋 最终启发式集:")
    for i, h in enumerate(final_population, 1):
        print(f"  {i}. {h['name']}")
        print(f"     平均性能: {h['avg_performance']:.4f}")

if __name__ == "__main__":
    main()

📋 步骤 1: 评估初始启发式
找到 4 个启发式算法:

test_data评估完成！结果汇总:
{'least_remaining_workload_lwr_d24d': [3193, 3887, 3101, 3780, 4391], 'longest_processing_time_lpt_e2c3': [2279, 3068, 2788, 3011, 3352], 'random_selection_b73a': [1497, 1601, 1721, 1869, 1699], 'shortest_processing_time_spt_4363': [2182, 3164, 2500, 2721, 3351]}

📋 步骤 2: 运行 EoH-S

🚀 EoH-S 算法开始
配置: 种群大小=5, 最大评估次数=100

✓ 初始化种群: 4 个启发式

📈 第 1 代进化 (已评估: 0/100)

🔄 开始Memetic Search，目标生成 5 个新启发式

  🔧 尝试 LS #1/5 (尝试 1/25)

✅ ls工作流开始!
    ❌ LS 失败: [Errno 2] No such file or directory: '\\src\\problems\\jssp\\task_description.txt'

  🔧 尝试 LS #1/5 (尝试 2/25)

✅ ls工作流开始!
    ❌ LS 失败: [Errno 2] No such file or directory: '\\src\\problems\\jssp\\task_description.txt'

  🔧 尝试 LS #1/5 (尝试 3/25)

✅ ls工作流开始!
    ❌ LS 失败: [Errno 2] No such file or directory: '\\src\\problems\\jssp\\task_description.txt'

  🔍 尝试 CS #1/5 (尝试 4/25)
✓ 选择的互补对: longest_processing_time_lpt_e2c3 和 random_selection_b73a
✓ 代码已保存到: \src\problems\jssp\heuristics\basic_heuristics\short

KeyboardInterrupt: 